# Compile the main simulator code using cython 

In [ ]:
!python ../setup.py build_ext --inplace

# Import all the relevant files 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import seaborn as sns
import pathos.multiprocessing

In [ ]:
import sys
sys.path.append('../')

#Importing scripts:

#Import relevant frames:
import common.cbgt as cbgt
import common.pipeline_creation as pl_creat

#Import plotting functions:
import common.plotting_functions as plt_func
import common.plotting_helper_functions as plt_help
import common.postprocessing_helpers as post_help

importlib.reload(plt_help)
importlib.reload(plt_func)
importlib.reload(post_help)

%load_ext autoreload
%autoreload 2
%reload_ext autoreload 

import warnings
warnings.simplefilter('ignore', category=FutureWarning)

# Choose the experiment and create the main pipeline

Choose the experiment to run, and define the number of choices, the number of simulations/thread to run and number of cores:

In [ ]:
#Choose the experiment:
experiment_choice = 'stop-signal'

if experiment_choice == 'stop-signal':
    import stopsignal.paramfile_stopsignal as paramfile
elif experiment_choice == 'n-choice':
    import nchoice.paramfile_nchoice as paramfile

number_of_choices = 2

#Choose which multiprocessing library to use:
use_library = "pathos" # "none" or "pathos" or "ray"


# how many simulations do you want to run? each simulation is executed as a separate thread.
num_sims = 1
num_cores = 7


#Call choose_pipeline with the pipeline object:
pl_creat.choose_pipeline(experiment_choice)
#Create the main pipeline:
pl = pl_creat.create_main_pipeline(runloop=True)

#Set a seed:
seed = np.random.randint(0,99999999,1)[0]
print(seed)

Define data and figures directories:

In [ ]:
data_dir = "../Data/"
figure_dir = "../Figures/"

# Modify cellular parameters as desired. 
### The paramfile has all the parameter dictionaries that can be modified. They are listed as below:
- celldefaults (neuronal parameters)
- d1defaults (dSPN parameters)
- d2defaults (iSPN parameters)
- dpmndefaults (dopamine related parameters)
- basestim (background input for the nuclei)
- popspecific (population specific parameters)
- receptordefaults (GABA, AMPA receptor parameters)
### The details of each of these dictionaries can be checked by simply typing paramfile.<parameter name> as also shown in the block below

In [ ]:
# list out the available parameter dictionaries:
dir(paramfile)

In [ ]:
# view (or edit) one of the parameter dictionaries:
paramfile.celldefaults

### To change a parameter, simply assign the desired value to the parameter
### eg. paramfile.celldefaults['C'] = 0.5

In [ ]:
# paramfile.celldefaults['C'] = 0.5

# Running the pipeline

### Define configuration parameter

In [ ]:
configuration = {
    'experimentchoice': experiment_choice,
    'inter_trial_interval': None,
    'thalamic_threshold': 30.,
    'movement_time': ['mean', 250], #default sampled from N(250,1.5), ["constant",250], ["mean",250]
    'choice_timeout': 300,
    
    'params': paramfile.celldefaults,  #neuron parameters
    'pops': paramfile.popspecific,    #population parameters
    'receps' : paramfile.receptordefaults, #receptor parameters
    'base' : paramfile.basestim,   #baseline stimulation parameters
    'dpmns' : paramfile.dpmndefaults,  #dopamine related parameters
    'dSPN_params' : paramfile.dSPNdefaults,     #dSPNs population related parameters
    'iSPN_params' : paramfile.iSPNdefaults,     #iSPNs population related parameters
    'channels' : pd.DataFrame([['left'], ['right']], columns=['action']), #action channels related parameters
    'number_of_choices': number_of_choices,
    'newpathways' : None,     #connectivity parameters
    'Q_support_params': None, #initialization of Q-values update
    'Q_df_set': pd.DataFrame([[0.5, 0.5]],columns=["left", "right"]), #initialized Q-values df  
    'n_trials': 1,            #number of trials
    'volatility': [None,"exact"], #frequency of changepoints
    'conflict': (1.0, 0),         #probability of the preferred choice
    'reward_mu': 1,               #mean for the magnitude of the reward
    'reward_std': 0.1,            #std for the magnitude of the reward
    'maxstim': 0.8,               #amplitude of the cortical input over base line
    'sustainedfraction': 0.75,
    
    #Stop signal
    'stop_signal_present': [True,True],
    'stop_signal_probability': [1., 1.],  #probability of trials that will get the stop-signal / list of trial numbers
    'stop_signal_amplitude': [0.6, 0.6],  #amplitude of the stop signal over base line
    'stop_signal_onset': [60.,60.],       #in ms
    'stop_signal_duration' : ["phase 0",165.], 
    'stop_signal_channel': ["all","left"], #"all" (all channels are given the stop signal) 
                                            #"any" (channel given the stop signal is chosen randomly)
                                            # [list of channels] == subset of channels given the stop signal
    'stop_signal_population':["STN","GPeA"],
    
    'record_variables':["stop_input"],
    
    
    #Opto signal
    'opt_signal_present': [False],
    'opt_signal_probability': [1.], # probability of trials that will get the optogenetic signal / list of trial numbers
    'opt_signal_amplitude': [.7],   #amplitude of the stop signal over base line
    'opt_signal_onset': [30.],      #in ms
    'opt_signal_duration': [150.],
    'opt_signal_channel': ["all"],  # "all" (all channels are given the stop signal) 
                                    #"any" (channel given the stop signal is chosen randomly)
                                    # [list of channels] == subset of channels given the stop signal
    
    'opt_signal_population':["iSPN"],
}

### Run the simulation

ExecutionManager class can take for 'use': 

- 'none', that corresponds to the singlethreaded mode;
- 'pathos', that corresponds to python's multiprocessing mode;
- 'ray', that corresponds to a multiprocessing library for python that operates on a client-server mode.

The default value is None (singlethreaded mode).

In [ ]:
results = cbgt.ExecutionManager(cores=num_cores,use=use_library).run([pl]*num_sims,[configuration]*num_sims)

# Results

List all the agent variables accessible: 

In [ ]:
results[0].keys()

In [ ]:
results[0]['stop_list_trials_list']

In [ ]:
results[0]['meaneff_GABA']

In [ ]:
results[0]['stop_signal_amplitude']

In [ ]:
experiment_choice

Extract all the relevant dataframes:

In [ ]:
firing_rates, rt_dist = plt_help.extract_relevant_frames(results,seed,experiment_choice)

In [ ]:
results[0]['popfreqs']

In [ ]:
firing_rates[0]

In [ ]:
recorded_variables = post_help.extract_recording_variables(results,results[0]['record_variables'],seed)

In [ ]:
recorded_variables['stop_input']#.melt(id_vars='Time(ms)')

Plot the recorded variable extracted. Figure is saved in the figure_dir previously specified:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
sns.set(style="white", font_scale=1.5)
sns.lineplot(x="Time(ms)",y="value",data=recorded_variables['stop_input'],hue='nuclei',lw=3.0,ax=ax)
ax.legend_.remove()
ax.spines.top.set_visible(False)
ax.spines.right.set_visible(False)
plt.tight_layout()
#fig.savefig(figure_dir+'stop_input.png')

Extract the data tables from the agent:

In [ ]:
datatables = cbgt.collateVariable(results,'datatables')
datatables[0]

Save the selected variables of results in the data_dir previously specified:

In [ ]:
postfix=""
cbgt.saveResults(results,data_dir+'network_data_'+str(seed)+'.pickle',['popfreqs','popdata','datatables'])

In [ ]:
firing_rates[0].to_csv(data_dir+"firing_rates_"+postfix+".csv")
rt_dist.to_csv(data_dir+"rt_dist_"+postfix+".csv")

Plot the firing rates extracted (figure handles are returned in fig_handles).
Figure is saved in the figure_dir previously specified:

In [ ]:
FR_fig_handles = plt_func.plot_fr(firing_rates,datatables,results,experiment_choice,True)
FR_fig_handles[0].savefig(figure_dir+"Example_FR_stopsignal.png",dpi=300)